# CS 121 Extra Programming assignment

This assignment is worth __40 bonus points__. You can work on this assignment in teams of __up to four students__. You have until __Saturday November 10th 11:59pm__ to submit it. There are no late days for this submission.

The point value of this assignment is probably not worth the time you will spend on it. Please work on it only if you:

1. Enjoy coding

2. Feel like you have a solid grasp of all other material in this course. (Otherwise you're better off spending your time on the theoretical components)


You will submit this assignment through gradescope. 




## Background: The BrainFuck Programming Language

The _BrainFuck Programming Language_ (see [Wikipedia](https://en.wikipedia.org/wiki/Brainfuck), henceforth BF for brevity and decency's sake) is a Turing complete programming language that has only eight simple commands.
The name of this language comes from the fact that BF programs tend to look like this:

```
>>,[>>,]
<<
[
[<<]>>>>[<<[>+<<+>-]>>
[>+<<<<[->]>[<]>>-]<<<
[[-]>>[>+<-]>>[<<<+>>>-]]
>>[[<+>-]>>]<]<<[>>+<<-]<<
]
>>>>[.>>]
```

(as you probably guessed immediately, this is  a program performs that performs the _bubble sort_; credit:  [Daniel Cristofani](http://www.hevanet.com/cristofd/brainfuck/bsort.b))

BF is Turing complete. In particular, the following is a BF interpreter in BF (taken from this [paper by Mazonka and Cristofani](https://arxiv.org/abs/cs/0311032v1) ):

```
>>>+[[-]>>[-]++>+>+++++++[<++++>>++<-]++>>+>+>+++++[>++>++++++<<-]+>>>,<++[[>[
->>]<[>>]<<-]<[<]<+>>[>]>[<+>-[[<+>-]>]<[[[-]<]++<-[<+++++++++>[<->-]>>]>>]]<<
]<]<[[<]>[[>]>>[>>]+[<<]<[<]<+>>-]>[>]+[->>]<<<<[[<<]<[<]+<<[+>+<<-[>-->+<<-[>
+<[>>+<<-]]]>[<+>-]<]++>>-->[>]>>[>>]]<<[>>+<[[<]<]>[[<<]<[<]+[-<+>>-[<<+>++>-
[<->[<<+>>-]]]<[>+<-]>]>[>]>]>[>>]>>]<<[>>+>>+>>]<<[->>>>>>>>]<<[>.>>>>>>>]<<[
>->>>>>]<<[>,>>>]<<[>+>]<<[+<<]<]
```


## The question

Your goal in this assignment is to implement the proof of the Cook Levin Theorem for BF.

Specifically, you will need to write a Python function `BF2NAND` such that if `P` is a string describing a BF program and `n` and `t` are integers then `BF2NAND(P,n,t)` will output the code of an $8n$-bit input and one-bit output NAND program $Q$ that satisfies the following. If

* `P` is a string which is the source code of a BF program which takes `n` bytes as input. We assume that first instructions of the program are to read these bytes into the memory array (this corresponds to having `n` copies of the pair of instructions `,>`  in the beginning of the program, and having no other `,` instructions in the program).

* `t` is an upper bound on the number of steps that the program `P` takes on every input of `n` bytes. Moreover, we assume that the program `P` outputs a single byte, which at the end of the program (which always happen within at most `t` steps) is at the first position of the memory array. 

Then the output of `BF2NAND(P,n,t)` will be a _NAND program_ $Q$ that takes $8n$ input bits and output a single bit, and has the property that for every $x\in \{0,1\}^{8n}$,  $Q(x)=1$ if and only if the BF program `P`, when given as input the $n$ bytes $x$, outputs a nonzero  byte. 

(We don't care what the output of `BF2NAND` is if it is given a string that does not code a valid BF program, or that encodes a program that does not satisfy the above assumptions.)

You will get 85% of the credit if you restrict yourself to only working for _balanced BF programs_. A BF program is _balanced_  if every loop in it (i.e., everything inside `[...]`) has an equal number of `<` and `>` operations, and so in the end of the loop the data pointer is in the same position that it started.

### Some extras

As in the last assignment, we might award bonus points to the team that manages to output the shortest NAND programs for a given BF input, though we urge you to first get a working sample before optimizing.  Once again you should remember Knuth's quote _"Premature optimization is the root of all evil."_ 

While you are guaranteed that no loop in the BF program will iterate more than `t` steps, you might use smarter analysis or benchmarking to get better bounds, and hence output smaller NAND code. For example, you can hardcode certain BF idioms such as `[-]`.

Since [Boaz's repository](https://github.com/boazbk/nandnotebooks/) already contains code that transforms NANDSAT to 3NAND and through there to 3SAT and INDEPENDENT SET, you can combine it with your code to transform a BF program `P` into a graph `G` and a number `k` such that `G` has an independent set of size `k` if and only if there is an input that causes `P` to output `1` within at most `t` steps. Feel free to post images of such graphs (with the BF code that they came from) on Piazza for other students to see and also enclode them in your submission. We might again give some bonus points to particularly impressive images.


## Detailed roblem Statement

The input will be a BF program $Q$ and parameters $n$ and $t$,  and the output will be a NAND program $P$ with $8n$ inputs and one output such that for every string $x$ of $8n$ bits (i.e., $n$ bytes), if $Q$ on input $x$ outputs $1$ within $t$ steps then $P(x)=1$ and otherwise $P(x)=0$. Since a BF program actually takes in a char (0-255) for every input, let us treat this as taking the bits into our NAND program with least significant bit first.


### BF specification

We will be using the original BF  standard. A specification can be found [here](https://github.com/brain-lang/brainfuck/blob/master/brainfuck.md). BF can be thought of as a tape, where each cell can hold an unsigned 8 bit integer. There are 8 basic instructions.

| Instruction | Description |
|-----|-----|
| `>` | move the pointer right |
| `<` | move the pointer left |
| `+` | increment the current cell |
| `-` | decrement the current cell |
| `.` | output the value of the current cell |
| `,` | **replace** the value of the current cell with input |
| `[` | jump to the **matching** `]` instruction if the current value is zero |
| `]` | jump to the **matching** `[` instruction if the current value is **not** zero |

We will treat the `.` as the output of our BF program and the items read in with `,` as the input of our BF program. All of the BF programs we will be testing will read in using `,` sequentially and outputting the first cell on the tape with `.`.


### Example

The main objective is to convert a BF program with $n$ inputs and $1$ output limited to $t$ operations into a NAND program that takes in $8n$ inputs and $1$ output, where the output of the NAND program is the least significant bit of the first BF output.

__Input__: 

* A BF program that just increments the 8 bit input by 1, and then returns the same. Sample code that does this is as follows (the BF interpreter ignores all of the symbols that aren't in the language) The program can be written as ```,+.```.

* Other parameters are $n = 1$, $t = 100$.

__Output__:

A possible output would just be the following NAND program:
```
 y[0] = XOR(x[0], 1)
 ```
 where we used `1` and `XOR` as syntactic sugar that corresponds to the actual NAND implementation of those operations.

## BF EVAL in Python

Before we convert BF  to NAND we still want a built-in evaluator that we know is correct. Some sample programs and outputs are provided.


In [163]:
!pip install bfi
!rm -rf nandprogramming
!git clone http://www.github.com/wfus/nandprogramming
!mv nandprogramming/nandutil.py .
%load nandutil.py

You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
Cloning into 'nandprogramming'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29 (delta 14), reused 17 (delta 8), pack-reused 0
Unpacking objects: 100% (29/29), done.


In [ ]:
import bfi
from nandutil import NANDProgram
from nandutil import EVAL
from nandutil import TRUTH

def int2str(*args):
    return "".join([str(chr(arg)) for arg in args])

def str2int(output):
    """Converts the string BF output to a list of integers"""
    int_st = [int(ord(char)) for char in output]
    return int_st

def run(bf_prog, *args):
    """Takes in a BF program and a list of integers and runs it. Returns the output
    as a list of integers."""
    bf_input = int2str(*args)
    ret = bfi.interpret(bf_prog, input_data=bf_input,
                        tape_size=1000, buffer_output=True)
    return str2int(ret)

def sanitize(bf_prog_string):
    """Sanitize a brainfuck program string by removing all unnecessary symbols"""
    vocabulary = [',', '.', '>', '<', '+', '-', '[', ']']
    return "".join([a for a in bf_prog_string if a in vocabulary])

In [ ]:
add2 = """
,>,<             read in two inputs
>>[-]            zero out 3rd cell
<                move to 2nd cell
[<+>>+<-]        ???
<                go back to 1st cell
>>[<+>-]         move 3rd cell to 2nd cell
<<               move back to first cell
.                print result
"""

notequal = """
x = first cell
y = second cell
tmp0 = third cell
tmp1 = fourth cell
checks if x is equal to y

,>,<
>>[-]                  zero out third cell
>[-]                   zero out fourth cell
<<<[>>>+<<<-]          ???
>[>>-<+<-]             ???
>[<+>-]                ???
>[<<<+>>>[-]]          ???
<<<.                   print out output
"""


read_2_print_2 = """
,>,<  read and go back to 0th cell
.>.   print out those two cells
"""

read_1_increment = """
,
+
.
"""

read_1_increment_2 = """
,
++
.
"""

In [166]:
print(run(add2, 2, 3))

[5]


In [167]:
print(run(notequal, 1, 1))
print(run(notequal, 123, 123))
print(run(notequal, 4, 10))

[0]
[0]
[1]


## BF to NAND

Now we need to actually convert BF program into a NAND program. We are guaranteed that we know the brainfuck program will get $n$ $8$ bit inputs, and return $1$ eight bit output. We want to construct a NAND program that takes in these $n$ $8$ bit inputs with least significant digit first, and returns a $1$ bit output corresponding to the least significant digit of the output.

In [2]:
def BF2NAND(bf_prog, n, t):
    # TODO: Implement BF  to nand where t is the number of steps
    # and t is the number of steps we must simulate the brainfuck
    # program with.
    raise NotImplementedError